In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# First, we will load and check the information about the data provided using head.

data = pd.read_excel('Payroll_Sneak_Peek_Data_Set.xlsx')
data.head()

,company_lead_id,created_dt_pst,source,states from lead form,switching,company_id,company_name,company_created_dt_pst,state,zip_code,...,suspended,suspended_date,suspended_reason,ran_payroll,first_payroll_date,first_invoice_month,first_invoice_amount,first_invoice_ees,first_invoice_ctrs,last_invoice_month
0,7757592465361942,2015-01-02,adwords,FL,Switching,7.757617e+15,Gerlach-Stark,2015-01-02,FL,33169,...,NaN,NaT,NaN,1,2015-01-30,2015-01-01,33.0,2.0,1.0,2015-03-01
1,7757592465361945,2015-01-02,NaN,ID,Switching,7.757617e+15,Leffler-Sauer,2015-02-11,ID,83340,...,NaN,NaT,NaN,1,2015-03-02,2015-03-01,77.0,13.0,1.0,2015-03-01
2,7757592465361992,2015-01-02,NaN,MS,New Company,7.757617e+15,Homenick-Legros,2015-01-21,MS,39470,...,NaN,NaT,NaN,1,2015-01-30,2015-01-01,37.0,3.0,1.0,2015-03-01
3,7757592465361997,2015-01-02,Direct,IL,Unknown,NaN,NaN,NaT,NaN,NaN,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT
4,7757592465362000,2015-01-02,NaN,MS,Unknown,NaN,NaN,NaT,NaN,NaN,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT


In [3]:
# Checking data inforamtion, datatypes, NULL values, etc.

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
company_lead_id           1000 non-null int64
created_dt_pst            1000 non-null datetime64[ns]
source                    983 non-null object
states from lead form     993 non-null object
switching                 1000 non-null object
company_id                836 non-null float64
company_name              836 non-null object
company_created_dt_pst    836 non-null datetime64[ns]
state                     530 non-null object
zip_code                  530 non-null object
account_status            836 non-null object
switching_status          836 non-null object
company_lead_id.1         836 non-null float64
lead_created_dt_pst       836 non-null datetime64[ns]
lead_source               831 non-null object
churned                   151 non-null float64
churned_date              2 non-null datetime64[ns]
suspended                 45 non-null float64
suspended_date            45 no

In [4]:
# Since there were few NULLS in source and in "states from lead form", we will fill them up with Mode value of the column

data['states from lead form'].replace({"":np.nan}, inplace=True)
data['states from lead form'].fillna(data['states from lead form'].mode().iloc[0], inplace = True)
data['source'].replace({"":np.nan}, inplace=True)
data['source'].fillna(data['source'].mode().iloc[0], inplace = True)
data['source'].value_counts()

adwords                    365
Inbound Referral           184
Direct                     123
facebook                   121
SEO                        119
bing                        35
Sales Inquiry               11
Data Sourcing                9
go-freshbooks                9
Resource Download            6
intuit                       4
freshbooks                   2
go-freshbooks-2014promo      2
y-combinator                 1
wework                       1
tsheets                      1
wheniwork                    1
500-startups                 1
Webinar                      1
Other                        1
go-tsheets-signup            1
kinhr                        1
Act-On Software              1
Name: source, dtype: int64

In [5]:
# Some values of "states from lead form" contains multiple states in single record.
# There is a trend that, value of state column is same as last mentioned state in multiple value record from "states from lead form" column.
# We will replace multiple NULL values of state column with last mentioned state of "states from lead form" value.

data['state'].replace({"":np.nan}, inplace=True)
data['state'].fillna(data['states from lead form'].str[-2:], inplace = True)
data['state'].value_counts()

CA    215
TX     97
FL     80
NY     75
IL     41
GA     40
WA     25
MA     24
AZ     23
OH     23
NC     23
PA     22
MI     21
OR     21
VA     20
NJ     19
CO     15
LA     15
IN     14
MO     14
MD     14
SC     13
AL     12
UT     10
WI      9
NV      9
IA      9
AR      8
TN      8
MN      8
OK      7
AK      7
KS      7
CT      7
KY      6
DC      5
MS      5
WY      4
NE      4
VT      3
RI      3
ME      3
ID      3
NH      2
NM      2
ND      1
HI      1
WV      1
DE      1
SD      1
Name: state, dtype: int64

In [6]:
# Now since we don't want multiple states in a single record of "states from lead form" column. Its time to clean it.

data['states from lead form'] = data['states from lead form'].str[0:2]
data['states from lead form'].value_counts()

CA    230
TX     91
FL     81
NY     67
IL     42
GA     37
MA     26
WA     25
AZ     25
NC     24
MI     22
PA     20
OR     19
OH     19
VA     18
NJ     18
IN     17
AL     17
LA     16
CO     16
MD     14
MO     13
SC     11
TN     10
WI      9
NV      9
IA      8
MN      8
UT      8
AR      8
OK      7
CT      7
AK      7
KS      7
DC      7
KY      6
MS      5
NE      4
ME      3
WY      3
VT      3
ID      2
NH      2
ND      2
NM      2
WV      1
RI      1
HI      1
DE      1
SD      1
Name: states from lead form, dtype: int64

In [7]:
data['switching'].value_counts()

New Company    602
Switching      234
Unknown        164
Name: switching, dtype: int64

In [8]:
# Since Unknonwn switching values has majority of NULL records, We will focuss on New Company and Switching records for now.
# dropna thresh condition is used to see the number of non NA column values. Therefore CleanData has no records.

UnknownData = data.loc[data['switching'].isin(['Unknown'])]
CleanData = UnknownData.dropna(thresh = 15)
CleanData.head()
UnknownData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164 entries, 3 to 991
Data columns (total 27 columns):
company_lead_id           164 non-null int64
created_dt_pst            164 non-null datetime64[ns]
source                    164 non-null object
states from lead form     164 non-null object
switching                 164 non-null object
company_id                0 non-null float64
company_name              0 non-null object
company_created_dt_pst    0 non-null datetime64[ns]
state                     164 non-null object
zip_code                  0 non-null object
account_status            0 non-null object
switching_status          0 non-null object
company_lead_id.1         0 non-null float64
lead_created_dt_pst       0 non-null datetime64[ns]
lead_source               0 non-null object
churned                   0 non-null float64
churned_date              0 non-null datetime64[ns]
suspended                 0 non-null float64
suspended_date            0 non-null datetime64[ns]
susp

In [9]:
# Selecting New Company and Switching records from switching column to focus more on these records

data = data.loc[data['switching'].isin(['New Company', 'Switching'])]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836 entries, 0 to 999
Data columns (total 27 columns):
company_lead_id           836 non-null int64
created_dt_pst            836 non-null datetime64[ns]
source                    836 non-null object
states from lead form     836 non-null object
switching                 836 non-null object
company_id                836 non-null float64
company_name              836 non-null object
company_created_dt_pst    836 non-null datetime64[ns]
state                     836 non-null object
zip_code                  530 non-null object
account_status            836 non-null object
switching_status          836 non-null object
company_lead_id.1         836 non-null float64
lead_created_dt_pst       836 non-null datetime64[ns]
lead_source               831 non-null object
churned                   151 non-null float64
churned_date              2 non-null datetime64[ns]
suspended                 45 non-null float64
suspended_date            45 non-nu

In [10]:
# Since zipcode doesn't add any value to the data and switching_status, company_lead_id, lead_created_dt_pst and lead source are duplicate column 

data.drop(['zip_code', 'switching_status', 'company_lead_id.1', 'lead_created_dt_pst', 'lead_source'], axis = 1, inplace = True)
data.head()

,company_lead_id,created_dt_pst,source,states from lead form,switching,company_id,company_name,company_created_dt_pst,state,account_status,...,suspended,suspended_date,suspended_reason,ran_payroll,first_payroll_date,first_invoice_month,first_invoice_amount,first_invoice_ees,first_invoice_ctrs,last_invoice_month
0,7757592465361942,2015-01-02,adwords,FL,Switching,7.757617e+15,Gerlach-Stark,2015-01-02,FL,Customer,...,NaN,NaT,NaN,1,2015-01-30,2015-01-01,33.0,2.0,1.0,2015-03-01
1,7757592465361945,2015-01-02,adwords,ID,Switching,7.757617e+15,Leffler-Sauer,2015-02-11,ID,Customer,...,NaN,NaT,NaN,1,2015-03-02,2015-03-01,77.0,13.0,1.0,2015-03-01
2,7757592465361992,2015-01-02,adwords,MS,New Company,7.757617e+15,Homenick-Legros,2015-01-21,MS,Customer,...,NaN,NaT,NaN,1,2015-01-30,2015-01-01,37.0,3.0,1.0,2015-03-01
5,7757592465362021,2015-01-02,adwords,CA,New Company,7.757617e+15,"Schoen, Gleichner and Marvin",2015-01-02,CA,Customer,...,NaN,NaT,NaN,1,2015-04-13,NaT,NaN,NaN,NaN,NaT
6,7757592465362047,2015-01-02,adwords,OR,New Company,7.757617e+15,D'Amore-Waelchi,2015-01-05,OR,Customer,...,NaN,NaT,NaN,1,2015-02-09,2015-02-01,37.0,1.0,2.0,2015-03-01


In [11]:
# We assumed Churned column value to be 1 incase of suspended or churned customer, 0 incase of current customer or blank incase of never customer
# Since Boarding is not considered as Suspended also Onboarding are not yet customers it can't be 0 therefore made it 2

def churn(data):
    if data['account_status'] == 'Customer':
        return 0
    elif ((data['account_status'] == 'Suspended Customer') | (data['account_status'] == 'Suspended In Setup')):
        return 1
    elif data['account_status'] == 'Onboarding':
        return 2
    
data['churned'] = data.apply(churn, axis = 1)
data['churned'].value_counts()

2    588
0    203
1     45
Name: churned, dtype: int64

In [12]:
# If customer is suspened in anyway then we would assume suspended flag as 1 else 0

def suspend(data):
    if ((data['account_status'] == 'Suspended Customer') | (data['account_status'] == 'Suspended In Setup')):
        return 1
    else:
        return 0

data['suspended'] = data.apply(suspend, axis = 1)
data['suspended'].value_counts()

0    791
1     45
Name: suspended, dtype: int64

In [13]:
# If the account status for a company is Customer or Suspened Cusomter then we would assume that we ran payroll for that company.

def payroll(data):
    if ((data['account_status'] == 'Customer') | (data['account_status'] == 'Suspended Customer')):
        return 1
    else:
        return 0

data['ran_payroll'] = data.apply(payroll, axis = 1)
data['ran_payroll'].value_counts()    

0    628
1    208
Name: ran_payroll, dtype: int64

In [14]:
# There is a relationship between churned_date and suspended_date

data.loc[data['churned'] == 1, ['account_status', 'churned', 'churned_date', 'suspended', 'suspended_date']]

,account_status,churned,churned_date,suspended,suspended_date
24,Suspended In Setup,1,NaT,1,2015-01-05
42,Suspended Customer,1,2015-02-01,1,2015-01-15
65,Suspended In Setup,1,NaT,1,2015-02-02
68,Suspended In Setup,1,NaT,1,2015-01-12
69,Suspended In Setup,1,NaT,1,2015-01-09
74,Suspended In Setup,1,NaT,1,2015-03-10
136,Suspended In Setup,1,NaT,1,2015-01-16
141,Suspended In Setup,1,NaT,1,2015-02-12
143,Suspended In Setup,1,NaT,1,2015-02-11
148,Suspended Customer,1,NaT,1,2015-03-13


In [15]:
# Since churned_date is NULL for suspended accounts, we will impute them with the value of suspended_date.
# A pattern is observed where churned_date is usually the next month start date of suspended_date, so I added monthbegin(1) to churn date 

for index,row in data.iterrows():
    churned = row['churned']
    churned_date = row['churned_date']
    suspended_date = row['suspended_date']
    if pd.isnull(churned_date) and (churned == 1):
        data.loc[index,'churned_date'] = data.loc[index,'suspended_date']
        data.loc[index,'churned_date'] = data.loc[index,'churned_date'] + pd.offsets.MonthBegin(1)

data.loc[data['churned'] == 1, ['account_status', 'churned', 'churned_date', 'suspended', 'suspended_date']]

,account_status,churned,churned_date,suspended,suspended_date
24,Suspended In Setup,1,2015-02-01,1,2015-01-05
42,Suspended Customer,1,2015-02-01,1,2015-01-15
65,Suspended In Setup,1,2015-03-01,1,2015-02-02
68,Suspended In Setup,1,2015-02-01,1,2015-01-12
69,Suspended In Setup,1,2015-02-01,1,2015-01-09
74,Suspended In Setup,1,2015-04-01,1,2015-03-10
136,Suspended In Setup,1,2015-02-01,1,2015-01-16
141,Suspended In Setup,1,2015-03-01,1,2015-02-12
143,Suspended In Setup,1,2015-03-01,1,2015-02-11
148,Suspended Customer,1,2015-04-01,1,2015-03-13


In [16]:
# Checking value of account status, ran payroll, first_payroll_date, first_payroll_month, last_invoice_month

data.loc[data['churned'] == 1, ['account_status', 'ran_payroll', 'first_payroll_date', 'first_payroll_month', 'last_invoice_month']]

C:\Users\sxh16\Anaconda3.5\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sxh16\Anaconda3.5\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,account_status,ran_payroll,first_payroll_date,first_payroll_month,last_invoice_month
24,Suspended In Setup,0,NaT,NaN,NaT
42,Suspended Customer,1,2015-01-15,NaN,2015-01-01
65,Suspended In Setup,0,NaT,NaN,NaT
68,Suspended In Setup,0,NaT,NaN,NaT
69,Suspended In Setup,0,NaT,NaN,NaT
74,Suspended In Setup,0,NaT,NaN,NaT
136,Suspended In Setup,0,NaT,NaN,NaT
141,Suspended In Setup,0,NaT,NaN,NaT
143,Suspended In Setup,0,NaT,NaN,NaT
148,Suspended Customer,1,2015-02-06,NaN,2015-03-01


In [17]:
# There are alot of NULLS for first_invoice_month, we imputed them using the value of first_payroll_date.
# Also there is a pattern where first_invoice_month is usually the first_date of month from first_payroll_date, so we will add monthbegin(-1) to first_invoice_month 

for index,row in data.iterrows():
    first_invoice_month = row['first_invoice_month']
    first_payroll_date = row['first_payroll_date']
    ran_payroll = row['ran_payroll']
    if  (ran_payroll == 1) and pd.isnull(first_invoice_month):
        data.loc[index,'first_invoice_month'] = data.loc[index,'first_payroll_date']
        data.loc[index,'first_invoice_month'] = data.loc[index,'first_invoice_month'] + pd.offsets.MonthBegin(-1)
        
data.loc[data['ran_payroll'] == 1, ['account_status', 'ran_payroll', 'first_payroll_date', 'first_invoice_month']]

,account_status,ran_payroll,first_payroll_date,first_invoice_month
0,Customer,1,2015-01-30,2015-01-01
1,Customer,1,2015-03-02,2015-03-01
2,Customer,1,2015-01-30,2015-01-01
5,Customer,1,2015-04-13,2015-04-01
6,Customer,1,2015-02-09,2015-02-01
8,Customer,1,2015-01-09,2015-01-01
13,Customer,1,2015-01-30,2015-01-01
15,Customer,1,2015-01-26,2015-01-01
17,Customer,1,2015-01-15,2015-01-01
18,Customer,1,2015-01-23,2015-01-01


In [18]:
# There are alot of NULLS for last_invoice_month, we imputed them using the value of first_invoice_month.

for index,row in data.iterrows():
    first_invoice_month = row['first_invoice_month']
    last_invoice_month = row['last_invoice_month']
    ran_payroll = row['ran_payroll']
    if (ran_payroll == 1) and pd.isnull(last_invoice_month):
        data.loc[index,'last_invoice_month'] = data.loc[index,'first_invoice_month']
        
data.loc[data['ran_payroll'] == 1, ['account_status', 'ran_payroll', 'first_payroll_date', 'first_invoice_month', 'last_invoice_month']]

,account_status,ran_payroll,first_payroll_date,first_invoice_month,last_invoice_month
0,Customer,1,2015-01-30,2015-01-01,2015-03-01
1,Customer,1,2015-03-02,2015-03-01,2015-03-01
2,Customer,1,2015-01-30,2015-01-01,2015-03-01
5,Customer,1,2015-04-13,2015-04-01,2015-04-01
6,Customer,1,2015-02-09,2015-02-01,2015-03-01
8,Customer,1,2015-01-09,2015-01-01,2015-03-01
13,Customer,1,2015-01-30,2015-01-01,2015-03-01
15,Customer,1,2015-01-26,2015-01-01,2015-03-01
17,Customer,1,2015-01-15,2015-01-01,2015-03-01
18,Customer,1,2015-01-23,2015-01-01,2015-03-01


In [19]:
data['first_invoice_ees'].value_counts()

1.0     66
2.0     34
3.0     16
4.0      8
8.0      6
7.0      6
5.0      5
6.0      3
10.0     1
16.0     1
15.0     1
26.0     1
12.0     1
19.0     1
13.0     1
Name: first_invoice_ees, dtype: int64

In [20]:
data['first_invoice_ctrs'].value_counts()

1.0     126
2.0      17
3.0       7
39.0      1
Name: first_invoice_ctrs, dtype: int64

In [21]:
# Since there were alot of NULL values in first_income_ctrs and first_income_ees, we will impute them randomly. 

fic_array = [1.0, 2.0, 3.0]

for index,row in data.iterrows():
    first_invoice_ctrs = row['first_invoice_ctrs']
    ran_payroll = row['ran_payroll']
    if ((pd.isnull(first_invoice_ctrs)) and (ran_payroll == 1)):
        randIndex = np.random.randint(len(fic_array))
        randvalue = fic_array[randIndex]
        #print(randvalue)
        data.loc[index,'first_invoice_ctrs'] = randvalue
    
fic_ees = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

for index,row in data.iterrows():
    first_invoice_ees = row['first_invoice_ees']
    ran_payroll = row['ran_payroll']
    if ((pd.isnull(first_invoice_ees)) and (ran_payroll == 1)):
        randIndex = np.random.randint(len(fic_ees))
        randvalue = fic_ees[randIndex]
        #print(randvalue)
        data.loc[index,'first_invoice_ees'] = randvalue
 
   
data.loc[data['ran_payroll'] == 1, ['account_status', 'ran_payroll', 'first_payroll_date', 'first_invoice_month', 'first_invoice_ees' , 'first_invoice_ctrs' , 'last_invoice_month']]

,account_status,ran_payroll,first_payroll_date,first_invoice_month,first_invoice_ees,first_invoice_ctrs,last_invoice_month
0,Customer,1,2015-01-30,2015-01-01,2.0,1.0,2015-03-01
1,Customer,1,2015-03-02,2015-03-01,13.0,1.0,2015-03-01
2,Customer,1,2015-01-30,2015-01-01,3.0,1.0,2015-03-01
5,Customer,1,2015-04-13,2015-04-01,2.0,3.0,2015-04-01
6,Customer,1,2015-02-09,2015-02-01,1.0,2.0,2015-03-01
8,Customer,1,2015-01-09,2015-01-01,7.0,3.0,2015-03-01
13,Customer,1,2015-01-30,2015-01-01,1.0,1.0,2015-03-01
15,Customer,1,2015-01-26,2015-01-01,1.0,1.0,2015-03-01
17,Customer,1,2015-01-15,2015-01-01,8.0,1.0,2015-03-01
18,Customer,1,2015-01-23,2015-01-01,2.0,1.0,2015-03-01


In [22]:
# Since the first_invoice_ees and first_invoice_ctrs are used to calculate first_invoice_month.
# Therefore we will divide data into X (first_invoice_ees, first_invoice_ctrs) where we have value of first_invoice_amount.
# X_pred (first_invoice_ees, first_invoice_ctrs) where we don't have value of first_invoice_amount.
# Y is the value of first_invoice_amount which we have; Y_pred is the value of first_invoice_amount which we will calculate using Regression.

X = data.loc[data['first_invoice_amount'].notnull() & (data['ran_payroll'] == 1), ['first_invoice_ees', 'first_invoice_ctrs']]
X_pred = data.loc[data['first_invoice_amount'].isnull() & (data['ran_payroll'] == 1),['first_invoice_ees', 'first_invoice_ctrs']]
Y = data.loc[data['first_invoice_amount'].notnull() & (data['ran_payroll'] == 1), ['first_invoice_amount']]

In [23]:
# Now we will fit X and Y on linear regression and use that to predict value of Y_pred. 

linreg = LinearRegression().fit(X, Y)
Y_pred = linreg.predict(X_pred)
Y_pred = pd.DataFrame(Y_pred)

In [24]:
X_pred.shape, Y_pred.shape

((59, 2), (59, 1))

In [25]:
# we will now reset index coz it will be used to join 2 dataframes (X_pred, Y_pred).

X_pred.reset_index (drop = True, inplace = True)

In [26]:
# We will join the join X_pred and Y_pred using left and right index

y_pred_new = X_pred.merge (Y_pred, left_index = True, right_index = True)
y_pred_new.head(2)

,first_invoice_ees,first_invoice_ctrs,0
0,2.0,3.0,71.100427
1,7.0,2.0,76.016861


In [27]:
# Since now we have got predicted value of first_invoice_amount (Y_pred) now we should fill it in the main dataframes.

for index_new,row_new in y_pred_new.iterrows():
    fie=row_new['first_invoice_ees']
    fic=row_new['first_invoice_ctrs']
    for index,row in data.iterrows():
        first_invoice_amount = row['first_invoice_amount']
        ran_payroll = row['ran_payroll']
        if ((ran_payroll == 1) & (pd.isnull(first_invoice_amount)) & (data.loc[index,'first_invoice_ctrs'] == fic) & (data.loc[index,'first_invoice_ees'] == fie)):
            data.loc[index, 'first_invoice_amount'] = y_pred_new.loc[index_new, 0]
            

data.loc[data['ran_payroll'] == 1]           

,company_lead_id,created_dt_pst,source,states from lead form,switching,company_id,company_name,company_created_dt_pst,state,account_status,...,suspended,suspended_date,suspended_reason,ran_payroll,first_payroll_date,first_invoice_month,first_invoice_amount,first_invoice_ees,first_invoice_ctrs,last_invoice_month
0,7757592465361942,2015-01-02,adwords,FL,Switching,7.757617e+15,Gerlach-Stark,2015-01-02,FL,Customer,...,0,NaT,NaN,1,2015-01-30,2015-01-01,33.000000,2.0,1.0,2015-03-01
1,7757592465361945,2015-01-02,adwords,ID,Switching,7.757617e+15,Leffler-Sauer,2015-02-11,ID,Customer,...,0,NaT,NaN,1,2015-03-02,2015-03-01,77.000000,13.0,1.0,2015-03-01
2,7757592465361992,2015-01-02,adwords,MS,New Company,7.757617e+15,Homenick-Legros,2015-01-21,MS,Customer,...,0,NaT,NaN,1,2015-01-30,2015-01-01,37.000000,3.0,1.0,2015-03-01
5,7757592465362021,2015-01-02,adwords,CA,New Company,7.757617e+15,"Schoen, Gleichner and Marvin",2015-01-02,CA,Customer,...,0,NaT,NaN,1,2015-04-13,2015-04-01,71.100427,2.0,3.0,2015-04-01
6,7757592465362047,2015-01-02,adwords,OR,New Company,7.757617e+15,D'Amore-Waelchi,2015-01-05,OR,Customer,...,0,NaT,NaN,1,2015-02-09,2015-02-01,37.000000,1.0,2.0,2015-03-01
8,7757592465362086,2015-01-02,Inbound Referral,GA,Switching,7.757617e+15,"Davis, O'Conner and Wyman",2015-01-02,GA,Customer,...,0,NaT,NaN,1,2015-01-09,2015-01-01,73.000000,7.0,3.0,2015-03-01
13,7757592465362208,2015-01-03,Direct,NY,New Company,7.757617e+15,"Dicki, Kuhlman and Stoltenberg",2015-01-03,NY,Customer,...,0,NaT,NaN,1,2015-01-30,2015-01-01,29.000000,1.0,1.0,2015-03-01
15,7757592465362291,2015-01-04,adwords,FL,New Company,7.757617e+15,Bernier Group,2015-01-04,FL,Customer,...,0,NaT,NaN,1,2015-01-26,2015-01-01,29.000000,1.0,1.0,2015-03-01
17,7757592465362295,2015-01-04,Direct,TX,Switching,7.757617e+15,Medhurst LLC,2015-01-04,TX,Customer,...,0,NaT,NaN,1,2015-01-15,2015-01-01,57.000000,8.0,1.0,2015-03-01
18,7757592465362303,2015-01-04,SEO,CA,New Company,7.757617e+15,Dooley-Kozey,2015-01-04,CA,Customer,...,0,NaT,NaN,1,2015-01-23,2015-01-01,33.000000,2.0,1.0,2015-03-01


In [28]:
# Writting data into Excel file, Sheet 1 for New Company and Switching data, Sheet 2 for Unknown data.

writer = pd.ExcelWriter('Payroll_Data_Analysis_Sneak_Peek.xlsx')
data.to_excel(writer,'New_Company & Switching')
UnknownData.to_excel(writer,'Unknown')
writer.save()

In [29]:
# Checking the value count of suspended reason

data['suspended_reason'].value_counts()

Switching payroll provider                                     7
Missing payroll features                                       6
No more employees to pay via payroll                           4
Fraud.                                                         3
My company was acquired / is shutting down                     2
Admin Request                                                  1
other                                                          1
Doesn't fit our methods                                        1
Not mine                                                       1
Unable to run payroll                                          1
Not ready for payroll services yet                             1
too confusing                                                  1
holding my $ for 6 weeks prior to paying IRS                   1
Price                                                          1
Never started services                                         1
MY ACCOUNT HAS BEEN DEACT